# GARBAGE DETECTION 

## GPU AND YOLO V8 IMPLEMENTATION

YOLOv8 uses PyTorch, so before installing it's always a good idea to check the system requirements needed. 

To use the GPU, we will need to understand what type of GPU does our computer have. If we have an NVIDIA GPU, then we use the CUDA toolkit, as well as the CUDNN libraries. If we have an AMD GPU, there is an open source website called Open CL.

It is important to check what version does PyTorch support for CUDA, as we will not be using the latest version (12.1 as of writing).

Here is a link to installing PyTorch: https://pytorch.org/get-started/locally/

Here is a link to the CUDA Toolkit Archive: https://developer.nvidia.com/cuda-toolkit-archive

This video explains the process pretty well: https://www.youtube.com/watch?v=Kp6c5v9iL84

Let us do a check if our GPU is recognized by PyTorch:

In [1]:
import torch
  
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")  


Is CUDA supported by this system? True
CUDA version: 11.1
ID of current CUDA device:0


As of writing, I have now made the change to upgrade my CUDA version from 11.1 to 11.8. 

In [1]:
import torch
  
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")  


Is CUDA supported by this system? True
CUDA version: 11.8
ID of current CUDA device:0


In [2]:
!nvidia-smi

Tue Apr 11 21:19:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 T...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8                4W /  N/A|      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Doing these tests and having the results as shown proves that the GPU is up and running. Please take note that this only applies if you have an NVIDIA GPU, so it may be best to consult other alternatives if you have an AMD GPU. 

Let us now set up the YOLOv8 repository. The key difference here is that everything is made convenient by just running one line of code. This code installs all the dependencies and libraries needed to run YOLOv8:

In [7]:
#!pip install ultralytics

**NOTE** 

By running this line of code, it actually installs the CPU version of PyTorch, so we will need to modify our code to have it install the GPU version. We only need to run the pip install one time.


To install the GPU version, we will need to go back to the PyTorch installation link (https://pytorch.org/get-started/locally/) and copy the code in our Anaconda Prompt. The difference is that we will just upgrade it. To do so, add "--upgrade " after install in your command prompt.

The upgrade code should look like this if you are upgrading to CUDA 11.8:

In [ ]:
#pip3 install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

What we recommend is to do pip install ultralytics first, and then install the GPU version of PyTorch to make things more efficient.

Here are the documentations for YOLOv8:

**GitHub Link**: https://github.com/ultralytics/ultralytics/blob/main/README.md

**Official Documentation**: https://docs.ultralytics.com/#yolo-a-brief-history

Let us now do a quick check to see if YOLOv8 is connected with our GPU:

In [2]:
from ultralytics import YOLO
!yolo checks

Ultralytics YOLOv8.0.58  Python-3.9.7 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti with Max-Q Design, 6144MiB)
Setup complete  (16 CPUs, 15.4 GB RAM, 345.6/458.4 GB disk)


YOLOv8 is now set-up to run with our local GPU with CUDA version 11.8.


## TRAINING THE MODEL

With everything set-up, we can now prepare to train the model. We can run this in 2 ways: Command Line Interface and Python style. Personally, CLI is easier to run and is less prone to errors as of writing. 

Before running the CLI (Command Line Interface), it is important to note the following:

1.) There is no need to zip the images. A thing of note is that the labels and images have to be in a separate folder. To do this you can create a folder named "labels" and change the directory in LabelImg to that folder. 

2.) A YAML file is needed, as this acts as our "repository", which contains the file paths of the train and validation images, the number of classes, as well as the class definitions. We can create this through code, or we can do it manually through a text editor. 



3.) Batch size may need to be reduced, as there are instances where we will have a runtime error due to the GPU having too low RAM, but this depends on how many VRAM does your GPU have.  

4.) The number of epochs may vary depending on the number of images, but for 500 images, 50-100 epochs is acceptable. There have been instances where there is early stoppage due to the accuracy not increasing, but we can always set a custom value by adding "patience = x". 

5.) To save the results in a folder, add "save=True" to your CLI or Python script, as there are instances where it did the results were not saved. 

Once we have these all ready, let us now train our model, which is essentially reduced to 1 line and can be run on either a notebook or on command prompt:


In [7]:
!yolo task=detect mode=train amp =False epochs=100 data=data_custom.yaml model=yolov8x.pt imgsz=640 batch=16 save=True

^C


Here is a guide on how to set-up YOLOv8 and briefly explains the whole process: https://www.youtube.com/watch?v=gRAyOPjQ9_s&t=1s

Optimizing and tuning the hyperparameters is a different story. Here is a link on the hyperparameters present for training the data: https://docs.ultralytics.com/modes/train/

## TESTING OUT THE MODEL USING VALIDATION IMAGES 

Now that we have our model, let us now do predictions on a validation dataset. 

We will now use a different mode, which is "predict", while our model is the one that was created in the "train" folder. 

You can also set your labels to only show its labels based on a minimum confidence percentage (e.g. you want the model to only show labels with at least 50% confidence). To do this, set conf = 0.5 (you can change this number depending on your needs).



In [3]:
yolo task=detect mode=predict model=best_535.pt source='C:/Users/Patrick/Desktop/CPSC_4830/Garbage_Detection_Project/yolov8/535images/val/images' save=True

Ultralytics YOLOv8.0.58  Python-3.9.7 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce GTX 1660 Ti with Max-Q Design, 6144MiB)
D:\Learnings\Anaconda\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs

image 1/10 C:\Users\Patrick\Desktop\CPSC_4830\Garbage_Detection_Project\255_images\val\images\IMG_0133.jpg: 640x480 4 non_organics, 80.4ms
image 2/10 C:\Users\Patrick\Desktop\CPSC_4830\Garbage_Detection_Project\255_images\val\images\IMG_0134.jpg: 640x480 3 non_organics, 76.4ms
image 3/10 C:\Users\Patrick\Desktop\CPSC_4830\Garbage_Detection_Project\255_images\val\images\IMG_0135.jpg: 640x480 3 non